In [1]:
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [2]:
# load the example dataset for diamonds
diamonds = sns.load_dataset("diamonds")

In [3]:
print(diamonds.head())

   carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75


In [4]:
spark = SparkSession.builder.appName("Diamonds").getOrCreate()

In [5]:
df = spark.createDataFrame(diamonds)

In [6]:
spark

In [7]:
print(diamonds.head())

   carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75


In [15]:
def preprocess_data(spark, data_path):
    # Load the dataset
    raw_data = data_path

    # Use 'species' column as the input for StringIndexer
    indexer = StringIndexer(inputCol="cut", outputCol="label")
    
    # Transform the data to include the 'label' column with indexed values
    indexed_data = indexer.fit(raw_data).transform(raw_data)

    # Specify feature columns to be combined into a single 'features' vector
    assembler = VectorAssembler(
        inputCols=["carat", "depth", "table", "x", "y", "z"],
        outputCol="features"
    )
    # Transform the data to include the new 'features' column with vectorized features
    vectorized_data = assembler.transform(indexed_data)
    
    # Select only the 'features' and 'label' columns
    final_data = vectorized_data.select("features", "label")

    # Split the data into training and test sets
    train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

    return train_data, test_data

In [16]:
train_data, test_data = preprocess_data(spark, df)

Py4JJavaError: An error occurred while calling o123.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 2.0 failed 1 times, most recent failure: Lost task 6.0 in stage 2.0 (TID 22) (AMALITECH-PC-11075 executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:310)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:322)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD.scala:322)
	at scala.collection.Iterator.foreach(Iterator.scala:94